# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_map_data = pd.read_csv("../output_data/cities.csv")
weather_map_data



,City,Lat,Lng,max temp,humidity,cloudiness,wind speed,Country,date
0,Butaritari,3.0707,172.7902,80.55,73,17,4.59,KI,1667877805
1,Ushuaia,-54.8000,-68.3000,55.02,54,20,5.75,AR,1667877530
2,Zapolyarnyy,69.4154,30.8136,27.68,94,86,3.51,RU,1667878365
3,Chokurdakh,70.6333,147.9167,-10.89,98,70,11.07,RU,1667878366
4,Albany,42.6001,-73.9662,48.63,58,4,10.40,US,1667877656
...,...,...,...,...,...,...,...,...,...
543,Aklavik,68.2191,-135.0107,0.73,85,100,10.83,CA,1667878004
544,Salym,60.0625,71.4789,19.54,96,98,7.23,RU,1667878103
545,Mikhaylovka,50.0600,43.2379,35.78,83,58,5.03,RU,1667878585
546,Tamsweg,47.1281,13.8110,41.68,84,46,4.09,AT,1667878586


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(g_key)

# Store the latitude and longitude in locations and store the humidity to be used as the weight
locations = weather_map_data[["Lat", "Lng"]].astype(float)
humidity = weather_map_data["humidity"].astype(float)

In [4]:
# Plot Heatmap 
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, 
                                 max_intensity = max(humidity), point_radius = 4)

# Add heat layer 
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities for ideal weather conditions and drop any null values
good_cities = weather_map_data.loc[(weather_map_data["max temp"] <= 82) & (weather_map_data["max temp"] >= 70) 
                                & (weather_map_data["wind speed"] <= 10) & (weather_map_data["cloudiness"] == 0)].dropna()

# Reset the index
good_cities = good_cities.reset_index(drop = True)

# Display results
good_cities

,City,Lat,Lng,max temp,humidity,cloudiness,wind speed,Country,date
0,Dwārka,22.2394,68.9678,80.73,52,0,6.67,IN,1667878369
1,Kirby,29.4633,-98.3856,75.81,80,0,9.22,US,1667878373
2,Cabo San Lucas,22.8909,-109.9124,73.51,75,0,8.05,MX,1667877950
3,Dakar,14.6937,-17.4441,77.13,88,0,0.00,SN,1667878262
4,Sandy Bay,-42.9130,147.3532,81.30,51,0,5.75,AU,1667878399
5,Mount Gambier,-37.8333,140.7667,80.17,36,0,2.68,AU,1667877842
6,Panabá,21.2833,-88.2667,72.72,93,0,6.53,MX,1667878480
7,Salalah,17.0151,54.0924,73.49,73,0,2.30,OM,1667878532
8,Uberaba,-19.7483,-47.9319,70.38,46,0,4.61,BR,1667878536
9,Verāval,20.9000,70.3667,81.55,51,0,9.55,IN,1667878547


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store a variable named hotel_df
hotel_df = good_cities

# Add a "Hotel_Name" column to DataFrame
hotel_df["hotel name"] = ""

# Display the results
hotel_df

,City,Lat,Lng,max temp,humidity,cloudiness,wind speed,Country,date,hotel name
0,Dwārka,22.2394,68.9678,80.73,52,0,6.67,IN,1667878369,
1,Kirby,29.4633,-98.3856,75.81,80,0,9.22,US,1667878373,
2,Cabo San Lucas,22.8909,-109.9124,73.51,75,0,8.05,MX,1667877950,
3,Dakar,14.6937,-17.4441,77.13,88,0,0.00,SN,1667878262,
4,Sandy Bay,-42.9130,147.3532,81.30,51,0,5.75,AU,1667878399,
5,Mount Gambier,-37.8333,140.7667,80.17,36,0,2.68,AU,1667877842,
6,Panabá,21.2833,-88.2667,72.72,93,0,6.53,MX,1667878480,
7,Salalah,17.0151,54.0924,73.49,73,0,2.30,OM,1667878532,
8,Uberaba,-19.7483,-47.9319,70.38,46,0,4.61,BR,1667878536,
9,Verāval,20.9000,70.3667,81.55,51,0,9.55,IN,1667878547,


In [8]:
# Set base url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key }

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude from the dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    response = requests.get(base_url, params = params).json()
    
    # Extract results
    results = response["results"]
    
    # Place results in Hotel Name column while handling errors
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"]= "Hotel not available"
      
        
# Display the results
hotel_df

,City,Lat,Lng,max temp,humidity,cloudiness,wind speed,Country,date,hotel name,Hotel Name
0,Dwārka,22.2394,68.9678,80.73,52,0,6.67,IN,1667878369,,"VITS Devbhumi, Dwarka"
1,Kirby,29.4633,-98.3856,75.81,80,0,9.22,US,1667878373,,Comfort Suites San Antonio Ft. Sam Houston/SAM...
2,Cabo San Lucas,22.8909,-109.9124,73.51,75,0,8.05,MX,1667877950,,"The Cape, A Thompson Hotel"
3,Dakar,14.6937,-17.4441,77.13,88,0,0.00,SN,1667878262,,Novotel Dakar
4,Sandy Bay,-42.9130,147.3532,81.30,51,0,5.75,AU,1667878399,,Hotel Grand Chancellor Hobart
5,Mount Gambier,-37.8333,140.7667,80.17,36,0,2.68,AU,1667877842,,The Commodore
6,Panabá,21.2833,-88.2667,72.72,93,0,6.53,MX,1667878480,,Hotel Amor
7,Salalah,17.0151,54.0924,73.49,73,0,2.30,OM,1667878532,,Salalah Gardens Hotel
8,Uberaba,-19.7483,-47.9319,70.38,46,0,4.61,BR,1667878536,,Tamareiras Prime Hotel
9,Verāval,20.9000,70.3667,81.55,51,0,9.55,IN,1667878547,,The Fern Residency Somnath


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))